In [ ]:
!pip install -q efficientnet

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import math
import tensorflow as tf
from tensorflow import keras
from sklearn import model_selection
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import efficientnet.tfkeras as efn
import h5py

In [ ]:
data_dir = Path('../input/landmark-recognition-2021')
test_data_dir = data_dir / 'test'
sample_file = data_dir / 'sample_submission.csv'

In [ ]:
sub = pd.read_csv(sample_file, index_col='id')

In [ ]:
def id_to_path(s, train=True):
    data_dir = train_data_dir if train else test_data_dir
    return data_dir / s[0] / s[1] / s[2] / f'{s}.jpg'

In [ ]:
input_size = (300, 300, 3)
batch_size = 32
seed = 42

In [ ]:
newsize = (300, 300)
# # x = np.asarray((Image.open(id_to_path('fd80b73a476ae8a1')).resize(newsize)))
# x = np.array(Image.open(id_to_path('00084cdf8f600d00',0)).resize(newsize))

In [ ]:
# x.shape

In [ ]:
# plt.imshow(x)

In [ ]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, x_set, y_set=None, batch_size=32):
        self.x , self.y = x_set, y_set
        self.batch_size = batch_size
        self.is_train = False if y_set is None else True
        
    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)
    
    def __getitem__(self, idx):
        batch_ids = self.x[idx * self.batch_size: (idx + 1) * self.batch_size]
        if self.y is not None:
            batch_y = self.y[idx * self.batch_size: (idx + 1) * self.batch_size]
        
#         list_x = [np.load(id_to_path(x, self.is_train))[::2] for x in batch_ids]
        newsize = (300, 300)
        list_x = np.array([np.asarray(Image.open(id_to_path(x, self.is_train)).resize(newsize)) for x in batch_ids])
#         list_x = np.concatenate(list_x, axis=0)
        batch_x = np.moveaxis(list_x,1,1)
        batch_x = batch_x.astype("float") / 255
        
        if self.is_train:
            return batch_x, batch_y
        else:
            return batch_x

In [ ]:
x1 = sub.index.values
test = DataGenerator(x1, batch_size=batch_size)

In [ ]:
model = tf.keras.models.load_model("../input/my-google-dataset/model_name.h5")

In [ ]:
# prediction = model.predict(test).flatten()
prediction = model.predict(test)

In [ ]:
predictions = np.argmax(prediction, axis=1)

In [ ]:
predictions.shape

In [ ]:
predictions

In [ ]:
scores = prediction.max(axis=1)

In [ ]:
scores.shape

In [ ]:
file = h5py.File('../input/my-google-dataset/data.h5', 'r')
index_numpy = file.get('dataset').value
file.close()

In [ ]:
index_numpy

In [ ]:
for i, index in enumerate(index_numpy):
    predictions[predictions==i] = index

In [ ]:
sub['landmarks'] = predictions
sub['scores'] = scores
sub['space'] = scores
sub['space'] = sub['space'].apply(lambda x: ' ')

sub['landmarks'] = sub['landmarks'].apply(lambda x: str(x))
sub['scores'] = sub['scores'].apply(lambda x: str("%.2f" % x))
sub['landmarks'] = sub['landmarks'] + sub['space'] + sub['scores']

sub.drop('scores', inplace=True, axis=1)
sub.drop('space', inplace=True, axis=1)

In [ ]:
sub.reset_index(inplace=True)

In [ ]:
sub.to_csv('submission.csv')

In [ ]:
sub